# TP 3 et 4 plan d'exécution de requêtes

date de modification: 01/02/2024

Nom: BOUZOURINE

Prénom: Hichem

Num étudiant: 21319982

# Préparation

installer DuckDB


In [ ]:
# duckdb est déjà installé sur les machines de colab
# !pip install duckdb --pre

Les lib nécessaires pour ce TP

In [ ]:
import duckdb

import sqlite3

import time
import pandas as pd

## Créer les données


On utilise la base de données du benchmark TPC-H.
DuckDB propose la procedure *dbgen* pour générer la base de données.
Les données sont stockées dans le fichier *tpch.db*


Comprendre le [schéma des données TPC-H](https://nuage.lip6.fr/s/RramHt5W3RomySs) (ou cf moodle).

La taille de la base est paramétrée par le scale factor *sf=0.2*


In [ ]:
# db = duckdb.connect(':memory:')
db = duckdb.connect('tpch.db')
db.execute("CALL dbgen(sf=0.1)")

## Consulter la BD

Il y a 8 tables. La commande `SHOW TABLES` retourne le nom des tables.

Note: la fonction  `.df()` ajoutée à la fin de l'expression sert à importer le résultat de la requête dans un dataframe pandas afin d'avoir un rendu "ergonomique" dans colab .

In [ ]:
db.execute("SHOW TABLES").df()

La commande `DESCRIBE` retourne une description des attributs d'une table.

Exemple affichant les attributs de la table *Lineitem* :

In [ ]:
db.execute("DESCRIBE lineitem").df()

La clause  `LIMIT N` est ajoutée à la fin d'une requête pour calculer seulement les N premiers tuples du résultat.

Exemple pour afficher 10 tuples de la table Lineitem :

In [ ]:
db.execute("SELECT * FROM lineitem LIMIT 10").df()

La commande `SUMMARIZE` donne un aperçu statistique des données d'une table.
Elle retourne pour chaque attribut, les bornes min et max du domaine, le nombre approximatif de valeurs uniques ainsi que la distribution décrite par la moyenne, l'écart type et les quartiles 25, 50 (médiane) et 75.

In [ ]:
db.execute("SUMMARIZE lineitem").df()

# Requêtes

On considère les requêtes suivantes qui sont des versions simplifiées d'une des requêtes du benchmark TPC-H :

In [ ]:
query1 = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""
# Query 1 : 12ms

query2 = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 50;
"""
# Query 2 : 11ms

query = query2

## Exécuter une requête

On définit la fonction `run_query` pour exécuter une requête et mesurer sa durée en millisecondes.

In [ ]:
# par défaut on exécute la requête dans DuckDB (db)
def run_query(q, connection=db):
  start = time.time()
  connection.execute(q).fetchall()
  end = time.time()
  return(f"{(end - start)*1000:.0f}ms")


Exécuter la première requête dans DuckDB

In [ ]:
run_query(query)

# Comparaison DuckDB versus SQLite

L'objectif est de comparer les performances de deux SGBD en mémoire :
DuckDB (très rapide) et SQLite (très populaire)

On veut montrer que DuckDB est plus rapide que SQLite pour traiter des requêtes "réalistes" de l'application TPC-H.


## Configuration de SQLite

Initialiser SQLite.

In [ ]:
# sqlite = sqlite3.connect(':memory:', check_same_thread=False)
!rm -f tpch_sqlite.db
sqlite = sqlite3.connect('tpch_sqlite.db', check_same_thread=False)

On peut transférer les données depuis DuckDB vers SQLite par l'intermédaire d'un dataFrame pandas.
*   On exporte chaque table de DuckDB vers un dataframe en utilisant  la commande `.df()`
*   Puis on utilise la fonction `to_sql` d'un dataframe pandas pour générer les instructions SQL permettant d'importer les données dans SQLite.


In [ ]:
table_list = db.execute("SHOW TABLES").fetchall()

# peut durer de 20s à  1 minute

for table in table_list:
  tname = table[0]
  table_data = db.table(tname).df()
  table_data.to_sql(tname, sqlite)
  # table_data.to_sql(tname, sqlite, if_exists='replace')


## Comparer les performances

On exécute la même requête dans les 2 SGBD: DuckDB et SQLite. Puis on compare les temps d'exécution.

In [ ]:
print("On exécute la requête:", query)

duckdb_result = run_query(query, db)
sqlite_result = run_query(query, sqlite)

print("Les durées d'exécution sont: ")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'SQLite': [sqlite_result]
})

On constate que DuckDB est environ 20 fois plus rapide que SQLite pour cette requête.

Afin de montrer l'efficacité de l'optimiseur de requêtes,
on demande à DuckDB d'exécuter la requête sans l'optimiser.

La commande  `PRAGMA disable_optimizer` désactive l'optimiseur de requête.
Puis on re-exécute la requête pour mesurer sa durée.

In [ ]:
db.execute("PRAGMA disable_optimizer")
duckdb_unoptimized_result = run_query(query)
db.execute("PRAGMA enable_optimizer")

pd.DataFrame.from_dict({
    'DuckDB': [duckdb_result],
    'DuckDB (sans optimisation de requête)': [duckdb_unoptimized_result],
    'SQLite': [sqlite_result]
})

On constate que la requête sans optimisation s'exécute beaucoup plus lentement (durée presque 5 fois plus longue!) que celle optimisée.

# Plan d'une requête
L'écart de durée entre la requête optimisée et celle non optimisée vient du fait qu'elles ont des plans d'exécution différents

On peut visualiser le **plan** d'exécution d'une requête en préfixant la requête par `EXPLAIN`.

La commande `EXPLAIN` ne fait qu'afficher le plan d'exécution ; elle n'exécute **pas** la requête.

Le plan affiché est appelé *plan physique* car il indique pour chaque opérateur du plan, le nom de l'algorithme utilisé:
*   HASH_GROUP_BY : regroupement par hachage puis agrégation
*   HASH_JOIN : jointure par hachage
*   FILTER : sélection en pipeline
*   SEQ_SCAN : lecture séquentielle d'une table

De plus le plan indique la cardinalité estimée **EC** pour certains opérateurs, en particulier pour les sélections et les jointures.

In [ ]:
def explain_query(query):
  print(db.execute("EXPLAIN " + query).fetchall()[0][1])

print(query)
explain_query(query)

Remarque : vous pourriez invoquer `PRAGMA explain_output='all'` pour afficher, en plus du plan physique, le plan logique avant son optimisation et le plan logique optimisé, mais ce n'est pas demandé dans ce TP.

# Rapport d'exécution d'un plan

On peut afficher un rapport **après** l'exécution d'un plan pour connaitre la quantité de données traitées par chaque opération du plan. On appelle cela le *profil* de l'exécution.

La fonction  `run_and_profile_query` active le mode profil afin de sauvegarder les informations de profil dans un fichier.
La requête est ensuite exécutée puis les informations de profil produites pendant l'exécution sont affichées.

On peut lire dans le profil, pour chaque opérateur, la durée écoulée, le nombre de tuples consommés en entrée ainsi que le nombre de tuples produits pour l'opérateur suivant.

Plus spécifiquement, l'opérateur de lecture séquentielle `SEQ_SCAN` (sequential scan) indique le nombre de tuples lus dans une table.
L'opérateur `HASH_GROUP_BY` indique le nombre de groupes créés.

Remarque: pour les opérateurs qui matérialisent leur sortie, le nombre de tuples produits représente la cardinalité du résultat intermédiaire.

Les cardinalités intermédiaires sont importantes car elles peuvent expliquer pourquoi un opérateur a pris beaucoup de temps pour être traité.
Dans de nombreuses situations, il est possible de réduire la cardinalité des résultats intermédiaires et modifiant l'ordre des opérations et la façon dont elles sont traitées.


In [ ]:
def run_and_profile_query(query):
  db.execute("PRAGMA enable_profiling")
  db.execute("PRAGMA profiling_output='out.log'")
  db.execute(query)
  db.execute("PRAGMA disable_profiling")
  with open('out.log', 'r') as f:
    output = f.read()
  print(output)

run_and_profile_query(query)


# Exercice 1 : Optimiser un première requête

Pour mieux comprendre le fonctionnement de l'optimiseur de requêtes, on propose d'optimiser la requête "à la main" en SQL.
Pour cela, on désactive l'optimiseur de requêtes avec la commande `PRAGMA disable_optimizer`. Cela signifie que le SGBD va traduire la requête SQL dans un plan qui sera "le plus proche possible" de l'ordre des insctructions écrites dans la requête SQL.


## Requête à optimiser

In [ ]:
query1_cross_product = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer,
    orders,
    lineitem
WHERE
    c_custkey=o_custkey
    AND l_orderkey=o_orderkey
    AND c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

# query2_cross_product = """
# SELECT
#     l_orderkey,
#     sum(l_extendedprice * (1 - l_discount)) AS revenue,
#     o_orderdate,
#     o_shippriority
# FROM
#     customer,
#     orders,
#     lineitem
# WHERE
#     c_custkey=o_custkey
#     AND l_orderkey=o_orderkey
#     AND c_mktsegment = 'BUILDING'
#     AND o_orderdate < CAST('1995-03-15' AS date)
#     AND l_shipdate > CAST('1995-03-15' AS date)
# GROUP BY
#     l_orderkey,
#     o_orderdate,
#     o_shippriority
# ORDER BY
#     revenue DESC,
#     o_orderdate
# LIMIT 50;
# """

Afficher le plan de la requête

In [ ]:
db.execute("PRAGMA disable_optimizer")
explain_query(query1_cross_product)

# Note: ne pas executer cette requête car son plan est tres lent!

On voit que le plan ne contient aucune jointure mais des produits cartésiens suivis de sélections. Or les produits cartésiens sont coûteux à calculer!

On évite d'exécuter cette requête pendant le TP car cela  prendrait trop de temps...



On remplace les produits cartésiens par des jointures en ajoutant  des `JOIN` dans la clause `FROM` (comme dans query1).

Cela rend la requête plus rapide et on peut l'exécuter pour mesurer sa durée.

In [ ]:
query_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_join)
run_query(query_join)

## a) Traiter les sélections avant les jointures

Prroposer d'autres écritures SQL équivalentes de la requête pour qu'elle soit traitée encore plus rapidement.

Utiliser des requêtes imbriquées pour pousser (ou non) les sélections et les projections.

Vérifier que la requête proposées est bien équivalente à la requête initiale
(avoir le même résultat est une condition nécessaire).



In [ ]:
query_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    customer
    JOIN (SELECT * FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date) ) AS o ON (c_custkey=o_custkey)
    JOIN (SELECT * FROM lineitem WHERE l_shipdate > CAST('1995-03-15' AS date) ) AS l ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_join)
run_query(query_join)


## b) Ordre des jointures

On peut voir que les tables sont jointes dans l'ordre Order,Customer, Lineitem

Re-ecrire la requête pour que les tables ne soient pas jointes dans des ordres différents. Est ce que cela change la durée de la requête ?

In [ ]:
query_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    lineitem
    JOIN (SELECT o_orderkey, o_custkey, o_shippriority FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date)) ON (l_orderkey=o_orderkey)
    JOIN (SELECT c_custkey FROM customer WHERE c_mktsegment = 'BUILDING' ) ON (c_custkey=o_custkey)
WHERE
    l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""

explain_query(query_join)
run_query(query_join)


## c) Index

Créer des index et montrer si cela a un effet sur la durée de la requête.

In [ ]:
index_orders_customer = "CREATE INDEX i_o_custkey ON orders (o_custkey);"
index_orders_date = "CREATE INDEX i_o_custkey ON orders (o_orderdate);"
index_customers = "CREATE INDEX i_c_mktsegment ON customer (c_mktsegment);"
index_lineitem = "CREATE INDEX i_l_shipdate ON linetime (l_shipdate);"
indexes = [index_orders_customer,index_orders_date,index_customers,index_lineitem]

map(run_query,indexes)


myquery_join = """
SELECT
    o_shippriority, count(*) as nb
FROM
    lineitem
    JOIN (SELECT o_orderkey, o_custkey, o_shippriority FROM orders WHERE o_orderdate < CAST('1995-03-15' AS date)) ON (l_orderkey=o_orderkey)
    JOIN (SELECT c_custkey FROM customer WHERE c_mktsegment = 'BUILDING' ) ON (c_custkey=o_custkey)
WHERE
    l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    o_shippriority
ORDER BY nb desc
LIMIT 50;
"""



explain_query(myquery_join)
run_query(myquery_join)



# Exercice 2 : Optimiser les requêtes TPCH

Optimiser chaque requête de TPC-H.
Vous pouvez les obtenir avec `SELECT * FROM tpch_queries()`


In [ ]:
queries = db.execute("SELECT * FROM tpch_queries()").fetchall()

# for id, sql in queries:
for id, sql in queries[:5]:
  print('### requete ', id, ':')
  print(sql)

In [ ]:
### requete  1 :
requete1_optimisee= """
SELECT
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,
    COUNT(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate <= '1998-09-02'::date
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
"""


### requete  2 optimisée :
requete2_optimisee = """
SELECT
    s.s_acctbal,
    s.s_name,
    n.n_name,
    p.p_partkey,
    p.p_mfgr,
    s.s_address,
    s.s_phone,
    s.s_comment
FROM
    part p
    JOIN partsupp ps ON p.p_partkey = ps.ps_partkey
    JOIN supplier s ON ps.ps_suppkey = s.s_suppkey
    JOIN nation n ON s.s_nationkey = n.n_nationkey
    JOIN region r ON n.n_regionkey = r.r_regionkey
    JOIN (
        SELECT
            ps_inner.ps_partkey,
            MIN(ps_inner.ps_supplycost) AS min_supplycost
        FROM
            partsupp ps_inner
            JOIN supplier s_inner ON ps_inner.ps_suppkey = s_inner.s_suppkey
            JOIN nation n_inner ON s_inner.s_nationkey = n_inner.n_nationkey
            JOIN region r_inner ON n_inner.n_regionkey = r_inner.r_regionkey
        WHERE
            r_inner.r_name = 'EUROPE'
        GROUP BY
            ps_inner.ps_partkey
    ) AS min_supply ON ps.ps_partkey = min_supply.ps_partkey AND ps.ps_supplycost = min_supply.min_supplycost
WHERE
    p.p_size = 15
    AND p.p_type LIKE '%BRASS'
ORDER BY
    s.s_acctbal DESC,
    n.n_name,
    s.s_name,
    p.p_partkey
LIMIT 100;
"""



### requete  3 optimisée :
requete3_optimisee = """
SELECT
    l.l_orderkey,
    SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue,
    o.o_orderdate,
    o.o_shippriority
FROM
    customer c
    JOIN orders o ON c.c_custkey = o.o_custkey
    JOIN lineitem l ON l.l_orderkey = o.o_orderkey
WHERE
    c.c_mktsegment = 'BUILDING'
    AND o.o_orderdate < '1995-03-15'::date
    AND l.l_shipdate > '1995-03-15'::date
GROUP BY
    l.l_orderkey,
    o.o_orderdate,
    o.o_shippriority
ORDER BY
    revenue DESC,
    o.o_orderdate
LIMIT 10;
"""



### requete  4 optimisée :
requete4_optimisee = """
SELECT
    o_orderpriority,
    COUNT(*) AS order_count
FROM
    orders o
WHERE
    o.o_orderdate >= '1993-07-01'::date
    AND o.o_orderdate < '1993-10-01'::date
    AND EXISTS (
        SELECT
            1
        FROM
            lineitem l
        WHERE
            l.l_orderkey = o.o_orderkey
            AND l.l_commitdate < l.l_receiptdate
    )
GROUP BY
    o.o_orderpriority
ORDER BY
    o.o_orderpriority;
"""



### requete  5 optimisée :
requete5_optimisee = """
SELECT
    n.n_name,
    SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue
FROM
    customer c
    JOIN orders o ON c.c_custkey = o.o_custkey
    JOIN lineitem l ON l.l_orderkey = o.o_orderkey
    JOIN supplier s ON l.l_suppkey = s.s_suppkey
    JOIN nation n ON s.s_nationkey = n.n_nationkey
    JOIN region r ON n.n_regionkey = r.r_regionkey
WHERE
    r.r_name = 'ASIA'
    AND o.o_orderdate >= '1994-01-01'::date
    AND o.o_orderdate < '1995-01-01'::date
GROUP BY
    n.n_name
ORDER BY
    revenue DESC;
"""


In [ ]:
explain_query(requete5_optimisee)

# Exercice 3 : Cardinalités

On considère le plan (optimisé) de la requête `query1`.

Expliquer comment est calculée chaque valeur de cardinalité estimée **EC**.

# Exercice 4 : Optimiser une requête avec 6 tables

Soit la requête suivante qui accède à 6 tables.

In [ ]:
query_6_tables = """
SELECT
    nation,
    o_year,
    sum(amount) AS sum_profit
FROM (
    SELECT
        n_name AS nation,
        extract(year FROM o_orderdate) AS o_year,
        l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity AS amount
    FROM
        part,
        supplier,
        lineitem,
        partsupp,
        orders,
        nation
    WHERE
        s_suppkey = l_suppkey
        AND ps_suppkey = l_suppkey
        AND ps_partkey = l_partkey
        AND p_partkey = l_partkey
        AND o_orderkey = l_orderkey
        AND s_nationkey = n_nationkey
        AND p_name LIKE '%green%') AS profit
GROUP BY
    nation,
    o_year
ORDER BY
    nation,
    o_year DESC;
"""

## a) Enumérer les ordres de jointure
Combien d'ordre de jointures sont possibles ?

In [ ]:
explain_query(query_6_tables)

Réponse: On remarque bien qu'il y a 6 jointures à faire.

## b) Requête optimale
 Re-*écrire* cette requête pour qu'elle s'exécute le plus rapidement possible.

In [ ]:
requete_optimale= """
SELECT
    n.n_name AS nation,
    EXTRACT(year FROM o.o_orderdate) AS o_year,
    SUM(l.l_extendedprice * (1 - l.l_discount) - ps.ps_supplycost * l.l_quantity) AS sum_profit
FROM
    lineitem l
    JOIN orders o ON l.l_orderkey = o.o_orderkey
    JOIN part p ON l.l_partkey = p.p_partkey AND p.p_name LIKE '%green%'
    JOIN partsupp ps ON l.l_suppkey = ps.ps_suppkey AND l.l_partkey = ps.ps_partkey
    JOIN supplier s ON l.l_suppkey = s.s_suppkey
    JOIN nation n ON s.s_nationkey = n.n_nationkey
GROUP BY
    n.n_name,
    o_year
ORDER BY
    n.n_name,
    o_year DESC;
"""
explain_query(requete_optimale)